# Error analysis of the global pre-processing workflow

Here we reproduce the error analysis shown in [Maussion et al. (2019)](https://www.geosci-model-dev.net/12/909/2019/) **for the pre-processing part only, and for the glacier directories (version 1.6 and 1.4)**. The error analysis of user runs needs a separate handling, see the [deal_with_errors](deal_with_errors.ipynb) notebook for more information.

## Get the files

We download the `glacier_statistics` files from the preprocessed directories folders, at the level 5. That is, we are going to count all errors that happened during the pre-processing chain.

In [ ]:
from oggm import utils
import pandas as pd
import seaborn as sns

We will start with a preprocessed directory for OGGM v1.6 (W5E5 with elev_bands):

In [ ]:
# This is W5E5 + elev_bands using OGGM v1.6, in the future there might be more preprocessed gdirs
url = ('https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/'
       'L3-L5_files/2023.1/centerlines/W5E5/RGI62/b_080/L5/summary/')

In [ ]:
# this can take some time
df = []
for rgi_reg in range(1, 19):
    fpath = utils.file_downloader(url + f'glacier_statistics_{rgi_reg:02d}.csv')
    df.append(pd.read_csv(fpath, index_col=0, low_memory=False))
df = pd.concat(df, sort=False).sort_index()

## Analyze the errors

In [ ]:
sns.countplot(y="error_task", data=df);

In [ ]:
"% area errors all sources: {:.2f}%".format(df.loc[~df['error_task'].isnull()].rgi_area_km2.sum() / df.rgi_area_km2.sum() * 100)

Most errors occur when making a gridded mask of the glacier outlines ('glacier_mask').

We now look at the errors that occur already before applying the climate tasks (i.e., before applying `mb_calibration_from_scalar_mb`):

In [ ]:
dfe = df.loc[~df['error_task'].isnull()]
dfe = dfe.loc[~dfe['error_task'].isin(['mb_calibration_from_scalar_mb'])]
"% area errors before climate: {:.2f}%".format(dfe.rgi_area_km2.sum() / df.rgi_area_km2.sum() * 100)

In the % of the area, much less glaciers fail due to the  option. 

In [ ]:
sns.countplot(y="error_task", data=dfe);

In [ ]:
# 15 largest glaciers
df.loc[~df['error_task'].isnull()].sort_values(by='rgi_area_km2', ascending=False)[['rgi_area_km2', 'error_task', 'error_msg']].iloc[:15]

## Example error comparison


- **CRU vs ERA5**:

In [ ]:
# This is CRU + centerlines. But you can try CRU+elev_bands, or ERA5+elev_bands, etc!
url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L3-L5_files/CRU/centerlines/qc3/pcp2.5/no_match/RGI62/b_040/L5/summary/'
# this can take some time
df_cru_v14 = []
# we don't look at RGI19 (Antarctic glaciers), because no climate available for CRU
for rgi_reg in range(1, 19):
    fpath = utils.file_downloader(url + f'glacier_statistics_{rgi_reg:02d}.csv')
    df_cru_v14.append(pd.read_csv(fpath, index_col=0, low_memory=False))
df_cru_v14 = pd.concat(df_cru_v14, sort=False).sort_index()

In [ ]:
# ERA5 uses a different precipitation factor 
url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.4/L3-L5_files/ERA5/centerlines/qc3/pcp1.6/no_match/RGI62/b_040/L5/summary/'
# this can take some time
df_era5_v14 = []
# we don't look at RGI19 (Antarctic glaciers), because no climate available for CRU
for rgi_reg in range(1, 19):
    fpath = utils.file_downloader(url + f'glacier_statistics_{rgi_reg:02d}.csv')
    df_era5_v14.append(pd.read_csv(fpath, index_col=0, low_memory=False))
df_era5_v14 = pd.concat(df_era5_v14, sort=False).sort_index()

In [ ]:
"% area errors all sources for ERA5 is {:.2f}% compared to CRU ({:.2f}%)".format(df_era5.loc[~df_era5['error_task'].isnull()].rgi_area_km2.sum() / df_era5.rgi_area_km2.sum() * 100,
                                                                              df.loc[~df['error_task'].isnull()].rgi_area_km2.sum() / df.rgi_area_km2.sum() * 100)

*more than three times less errors from the climate tasks occur when using ERA5 than when using CRU* !

- **centerlines vs elevation bands**:

In [ ]:
# CRU elevation bands
url = ('https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/'
       'L3-L5_files/2023.1/elev_bands/W5E5/RGI62/b_080/L5/summary/')
# this can take some time
df_elev = []
# we don't look at RGI19 (Antarctic glaciers), because no climate available for CRU
for rgi_reg in range(1, 19):
    fpath = utils.file_downloader(url + f'glacier_statistics_{rgi_reg:02d}.csv')
    df_elev.append(pd.read_csv(fpath, index_col=0, low_memory=False))
df_elev = pd.concat(df_elev, sort=False).sort_index()

In [ ]:
"% area errors all sources for elevation band flowlines is {:.2f}% compared to centerlines ({:.2f}%) with W5E5".format(df_elev.loc[~df_elev['error_task'].isnull()].rgi_area_km2.sum() / df_elev.rgi_area_km2.sum() * 100,
                                                                              df.loc[~df['error_task'].isnull()].rgi_area_km2.sum() / df.rgi_area_km2.sum() * 100)

In [ ]:
df_elev.error_msg.dropna().unique()

*less errors occur when using elevation band flowlines than when using centerlines !*

## What's next?

- A more detailed analysis about the type, amount and relative failing glacier area (in total and per RGI region) can be found in [this error analysis jupyter notebook](https://nbviewer.org/urls/cluster.klima.uni-bremen.de/~lschuster/error_analysis/error_analysis_v1.ipynb?flush_cache=true). It also includes an error analysis for different [MB calibration and climate quality check methods](https://nbviewer.org/urls/cluster.klima.uni-bremen.de/~lschuster/error_analysis/error_analysis_v1.ipynb?flush_cache=true#Analysis-for-Level-5-pre-processing-directories!).
- If you are interested in how the “common” non-failing glaciers differ in terms of historical volume change, total mass change and specific mass balance between different pre-processed glacier directories, you can check out [this jupyter notebook](https://nbviewer.org/urls/cluster.klima.uni-bremen.de/~lschuster/error_analysis/working_glacier_gdirs_comparison.ipynb?flush_cache=true).
- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)